In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def detect_pose(frame):
    # Convert the image to RGB
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Get pose landmarks
    result = pose.process(rgb_image)
    
    # Extract keypoints
    keypoints = []
    if result.pose_landmarks:
        for lm in result.pose_landmarks.landmark:
            keypoints.append((lm.x, lm.y, lm.z, lm.visibility))  # x, y, z, visibility
    
    return keypoints, result.pose_landmarks

# Test on webcam feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    keypoints, landmarks = detect_pose(frame)
    
    # Draw landmarks on the frame
    if landmarks:
        mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose Detection', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

/Users/suryanshpatel/Projects/exercise_correction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-10-22 18:37:01.831 Python[57212:7121731] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-22 18:37:01.832 Python[57212:7121731] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [7]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples (people)
num_samples = 1000

# Function to simulate keypoints data with different patterns based on pain levels
def generate_keypoints(pain_level):
    base_keypoints = np.random.rand(33, 3)  # Random base keypoints for a person
    # Introduce distortions based on pain level
    if pain_level < 3:  # Low pain (normal posture)
        return base_keypoints
    elif pain_level < 7:  # Medium pain (slight distortions in posture)
        distortion = np.random.normal(0, 0.05, size=(33, 3))  # Small distortion
        return base_keypoints + distortion
    else:  # High pain (strong distortions in posture)
        distortion = np.random.normal(0, 0.1, size=(33, 3))  # Larger distortion
        return base_keypoints + distortion

# Generate keypoints data and corresponding pain levels
pain_labels = np.random.randint(0, 11, size=num_samples)  # Pain levels between 0 and 10
keypoints_data = [generate_keypoints(pain_level) for pain_level in pain_labels]

# Flatten the keypoints into a single list for each person
X = [kp.flatten() for kp in keypoints_data]
y = pain_labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate model
accuracy = clf.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.12


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Assume clf is your pre-trained pain estimation model
# Load your pre-trained pain model
# For example: clf = joblib.load('pain_model.pkl')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def detect_pose(frame):
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_image)
    
    keypoints = []
    if result.pose_landmarks:
        for lm in result.pose_landmarks.landmark:
            keypoints.append((lm.x, lm.y, lm.z, lm.visibility))
    
    return keypoints, result.pose_landmarks

def estimate_pain(keypoints):
    if keypoints:
        # Flatten keypoints for the model
        flat_keypoints = np.array(keypoints).flatten().reshape(1, -1)
        # Predict pain level
        pain_level = clf.predict(flat_keypoints)
        return pain_level[0]
    return None

# Test on webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    keypoints, landmarks = detect_pose(frame)
    
    if landmarks:
        mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Estimate pain level
        pain_level = estimate_pain(keypoints)
        if pain_level is not None:
            cv2.putText(frame, f"Pain Level: {pain_level}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Pose Detection and Pain Estimation', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define a simple neural network model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(len(X_train[0]),)),  # Input: flattened keypoints
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')  # Regression for pain level
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32, validation_data=(np.array(X_test), np.array(y_test)))

# Save the model
model.save('pain_estimation_model.h5')


Epoch 1/10


2024-10-22 18:43:36.428876: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


25/25 [==============================] - 0s 3ms/step - loss: 15.1445 - val_loss: 9.9703
Epoch 2/10
25/25 [==============================] - 0s 927us/step - loss: 10.1927 - val_loss: 9.9394
Epoch 3/10
25/25 [==============================] - 0s 774us/step - loss: 10.0247 - val_loss: 10.0884
Epoch 4/10
25/25 [==============================] - 0s 795us/step - loss: 9.8940 - val_loss: 10.0071
Epoch 5/10
25/25 [==============================] - 0s 782us/step - loss: 9.7452 - val_loss: 10.0854
Epoch 6/10
25/25 [==============================] - 0s 853us/step - loss: 9.5784 - val_loss: 10.1288
Epoch 7/10
25/25 [==============================] - 0s 773us/step - loss: 9.3479 - val_loss: 10.1072
Epoch 8/10
25/25 [==============================] - 0s 859us/step - loss: 9.1478 - val_loss: 10.1372
Epoch 9/10
25/25 [==============================] - 0s 839us/step - loss: 9.0525 - val_loss: 10.2729
Epoch 10/10
25/25 [==============================] - 0s 902us/step - loss: 8.9153 - val_loss: 10.8693


: 

In [ ]:
import streamlit as st
from torchvision import models, transforms
from PIL import Image
import torch

# Load model
model = models.resnet18(pretrained=True)
model.eval()

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type="jpg")
if uploaded_file is not None:
    img = Image.open(uploaded_file)
    st.image(img, caption='Uploaded Image', use_column_width=True)
    
    # Preprocess image
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)
    
    # Prediction
    with torch.no_grad():
        output = model(input_batch)
    st.write(f'Predicted class: {torch.argmax(output[0])}')


In [ ]:
import streamlit as st
from transformers import pipeline

# Load model
summarizer = pipeline("summarization")

# Get text input
text = st.text_area("lower back is in pain")

# Summarize and display result
if text:
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    st.write(summary[0]['summary_text'])
